<a href="https://colab.research.google.com/github/andrePankraz/speech_service/blob/main/notebooks/NLLB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Translation - No Language Left Behind (NLLB)
The following Notebook can translate text between 200 languages. It's based on the Meta model [NLLB](https://ai.facebook.com/research/no-language-left-behind/).

# Set-up environment
We need following packages:

*   [transformers](https://github.com/huggingface/transformers) for NLLB model
*   bitsandbytes & accelereate for [8bit model inference](https://huggingface.co/blog/hf-bitsandbytes-integration)
*   [sentence_cleaner_splitter](https://github.com/facebookresearch/LASER/tree/main/utils) from project [LASER](https://github.com/facebookresearch/LASER) for sentence splitting

In [1]:
!pip install --quiet bitsandbytes
!pip install --quiet git+https://github.com/huggingface/transformers.git # Install latest version of transformers
!pip install --quiet accelerate
!pip install --quiet sentence_cleaner_splitter@git+https://github.com/facebookresearch/LASER.git#subdirectory=utils

     |████████████████████████████████| 62.5 MB 1.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 7.6 MB 4.7 MB/s 
     |████████████████████████████████| 163 kB 68.8 MB/s 
     |████████████████████████████████| 148 kB 4.8 MB/s 
     |████████████████████████████████| 40 kB 3.5 MB/s 
     |████████████████████████████████| 44 kB 2.9 MB/s 
     |████████████████████████████████| 70 kB 7.0 MB/s 
     |████████████████████████████████| 8.1 MB 41.1 MB/s 
     |████████████████████████████████| 2.6 MB 44.5 MB/s 
     |████████████████████████████████| 883 kB 68.1 MB/s 
     |████████████████████████████████| 212 kB 69.8 MB/s 
     |████████████████████████████████| 175 kB 74.8 MB/s 
     |████████████████████████████████| 2.8 MB 43.9 MB/s 
     |████████████████████████████████| 9.6 MB 42.7 MB/s 
     |████████████████████████████████| 965 kB 62.4 MB/s 


# Check GPU
The following experiments can be run without a GPU, but it will take much longer!

See Colab Menu: Runtime / Change type.

Check if GPU available:

In [2]:
!nvidia-smi

Thu Nov  3 14:29:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Set-up AI-model for translation
We need a pre-trained tokenizer and a model, that are downloaded from [Hugging Faces](https://huggingface.co/models?sort=downloads&search=facebook%2Fnllb). 

Multiple model sizes are available, to adapt to the GPU VRAM constraints. Only the big models provide enough quality, the smaller are just for quick tests.

With the param "load_in_8bit=True" you can also load bigger models, but they are slow at inference time. See [bitsandbytes](https://huggingface.co/blog/hf-bitsandbytes-integration).

In [3]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

# Load model NLLB
# facebook/nllb-200-distilled-600M, facebook/nllb-200-distilled-1.3B, facebook/nllb-200-3.3B
# VRAM at least: 4 | 8 | 16 GB VRAM
model_id = 'facebook/nllb-200-distilled-1.3B'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map="auto", load_in_8bit=True)

Downloading:   0%|          | 0.00/564 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/808 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/6.93G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.55G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.10G [00:00<?, ?B/s]

A source and a target language are to be provided to the translation pipeline. The supported language IDs are visible with this map from project [LASER](https://github.com/facebookresearch/LASER), which has been used for NLLB training.

In [4]:
# Optional output...
from sentence_cleaner_splitter.sentence_split import split_lang_code_map

split_lang_code_map

| 2022-11-03 14:38:24,840 | INFO | khmer-nltk | Loaded model from /usr/local/lib/python3.7/dist-packages/khmernltk/word_tokenize/sklearn_crf_ner_10000.sav |
INFO:khmer-nltk:Loaded model from /usr/local/lib/python3.7/dist-packages/khmernltk/word_tokenize/sklearn_crf_ner_10000.sav
| 2022-11-03 14:38:24,879 | INFO | khmer-nltk | Loaded model from /usr/local/lib/python3.7/dist-packages/khmernltk/pos_tag/sklearn_crf_pos_alt_0.9846.sav |
INFO:khmer-nltk:Loaded model from /usr/local/lib/python3.7/dist-packages/khmernltk/pos_tag/sklearn_crf_pos_alt_0.9846.sav


{'ace_Arab': 'ace_Arab',
 'ace_Latn': 'ace_Latn',
 'acm_Arab': 'acm',
 'acq_Arab': 'acq',
 'aeb_Arab': 'aeb',
 'afr_Latn': 'afr',
 'ajp_Arab': 'ajp',
 'aka_Latn': 'aka',
 'amh_Ethi': 'amh',
 'apc_Arab': 'apc',
 'arb_Arab': 'ara_Arab',
 'arb_Latn': 'ara_Latn',
 'ars_Arab': 'ars',
 'ary_Arab': 'ary',
 'arz_Arab': 'arz',
 'asm_Beng': 'asm',
 'ast_Latn': 'ast',
 'awa_Deva': 'awa',
 'ayr_Latn': 'ayr',
 'azb_Arab': 'azb',
 'azj_Latn': 'azj',
 'bak_Cyrl': 'bak',
 'bam_Latn': 'bam',
 'ban_Latn': 'ban',
 'bel_Cyrl': 'bel',
 'bem_Latn': 'bem',
 'ben_Beng': 'ben',
 'bho_Deva': 'bho',
 'bjn_Arab': 'bjn_Arab',
 'bjn_Latn': 'bjn_Latn',
 'bod_Tibt': 'bod',
 'bos_Latn': 'bos',
 'bug_Latn': 'bug',
 'bul_Cyrl': 'bul',
 'cat_Latn': 'cat',
 'ceb_Latn': 'ceb',
 'ces_Latn': 'ces',
 'cjk_Latn': 'cjk',
 'ckb_Arab': 'ckb',
 'crh_Latn': 'crh_Latn',
 'cym_Latn': 'cym',
 'dan_Latn': 'dan',
 'deu_Latn': 'deu',
 'dik_Latn': 'dik',
 'diq_Latn': 'diq',
 'dyu_Latn': 'dyu',
 'dzo_Tibt': 'dzo',
 'ell_Grek': 'ell',
 'eng

In [5]:
src_lang='deu_Latn'
tgt_lang='rus_Cyrl'

src_text = '''
Menschheitstraum
Das Verstehen einer Sprache, ohne sie gelernt zu haben, ist ein alter Menschheitstraum (Turmbau zu Babel, J. Bechers numerische Interlingua, Timerio, Babelfisch, Pfingstwunder, Science-Fiction-Geschichten). Die Erfindung der Computer in Kombination mit der Beschäftigung mit dem Phänomen Sprache als wissenschaftliche Disziplin (Sprachwissenschaft) hat zum ersten Mal einen konkreten Weg zur Erfüllung dieses Traums geöffnet.

Geschichte
Bis zum heutigen Tag hat das militärische Interesse den Weg der MÜ entscheidend geprägt. Eines der frühesten Projekte war ein Russisch-Englisch-Übersetzungsprogramm für das US-Militär. Trotz seiner anekdotenhaft schlechten Qualität genoss das Programm hohe Popularität unter US-Militärs, die sich zum ersten Mal ohne den Umweg über Dritte (Dolmetscher und Übersetzer) selbst zumindest einen Eindruck vom Inhalt russischer Dokumente verschaffen konnten.

Der 1966 für das Verteidigungsministerium der Vereinigten Staaten erstellte ALPAC-Bericht[1] bescheinigte der MÜ grundsätzliche Unrealisierbarkeit und brachte mit einem Schlag die Forschung für fast 20 Jahre praktisch ganz zum Erliegen. Erst in den 1980er Jahren begannen Elektrokonzerne wie die Siemens AG (Metal-Projekt) erneut mit der Forschung. Zu diesen Vorhaben zählt auch die Forschungsarbeit im Sonderforschungsbereich „Elektronische Sprachforschung“ an der Universität des Saarlandes. Hier wurde das System „SUSY“ entwickelt, das in der Lage war, aus dem Deutschen und ins Deutsche zu übersetzen.[2] Ein weiteres System des Sonderforschungsbereichs war ASCOF, in dem neben morpho-syntaktischen auch semantische Informationen für die Übersetzung herangezogen wurden.[3] In der gleichen Zeit initiierte die japanische Regierung das Fünfte-Generation-Projekt, bei dem MÜ vom Englischen ins Japanische zunächst auf der Basis der Programmiersprache Prolog implementiert wurde. Die enge Zusammenarbeit zwischen Universitäten, Elektrokonzernen und Regierung führte zu den weltweit ersten kommerziellen MÜ-Programmen für PCs und hat Japan in die Führungsposition der MÜ-Forschung weltweit gebracht. In den 1990er Jahren lief in Deutschland das BMBF-Leitprojekt Verbmobil, dessen Ziel es war, deutsche, englische und japanische gesprochene Dialogsprache zu dolmetschen. Das Verbmobil-System sollte gesprochene Spontansprache erkennen, die Eingabe analysieren, übersetzen, einen Satz erzeugen und ihn aussprechen.[4]

In den 2000er Jahren kamen vermehrt statistische Verfahren zum Einsatz. So bietet Google seit 2006 ein statistisches Übersetzungssystem an.[5] Auch regelbasierte Ansätze wurden weiterentwickelt. Eines der bekanntesten Forschungsprojekte dieser Art ist die freie Software Apertium, die von der spanischen Regierung und der Regierung von Katalonien finanziert und an der Universität Alicante weiterentwickelt wird.

Der Stand der MÜ im Jahr 2010 wurde von vielen Menschen als unbefriedigend bewertet. Grundsätzlich versteht die Wissenschaft menschliche Sprache aber noch unzureichend. Die meisten Sprachwissenschaftler gingen gar davon aus, dass maschineller Übersetzung ohne über das reine Sprachverständnis weit hinausgehende Kompetenzen automatischer Systeme grundsätzliche Grenzen gesetzt sind, da viele Übersetzungen zudem große Mengen an konzeptuellem Wissen, Metawissen sowie Kenntnisse über die Konstitution menschlicher Umwelt allgemein und über die Konventionen sozialer Interaktion erfordern.

Seit dem Jahr 2016 werden für Übersetzungsprogramme zunehmend künstliche neuronale Netze, d. h. künstliche Intelligenzen eingesetzt, wodurch der Fortschritt rasant zunahm. Beispiele sind DeepL, Google Übersetzer, Yandex.Translate sowie der Bing Translator, die fortan deutlich bessere Ergebnisse erzielten.[6]

Im März 2018 teilte Microsoft mit, durch eine KI Chinesisch-Englisch-Übersetzungen mit der Qualität eines professionellen menschlichen Übersetzers zu erreichen. Das sei ein Durchbruch bei der maschinellen Übersetzung, den Microsoft nicht so früh erwartet habe.[7][8]

Der Bedarf an MÜ-Anwendungen steigt weiter:

Viele Texte sind heute digital verfügbar (also leicht für den Computer zu verarbeiten).
Die Globalisierung erfordert die Übertragung von immer mehr Texten in immer mehr Sprachen (der Markt für Übersetzung verdoppelt sich alle vier Jahre), während die Popularität des Berufs des Übersetzers/Dolmetschers stagniert.
Gerade von nur wenigen Westeuropäern/Amerikanern gesprochene beziehungsweise für diese schwierig zu erlernende Sprachen aus Regionen, deren Bewohner ihrerseits kaum westliche Sprachen sprechen, werden immer wichtiger:
kommerziell wichtig: die ostasiatischen Sprachen Chinesisch, Koreanisch und Japanisch; sowie Thai.
militärisch wichtig: Sprachen der internationalen Konfliktregionen, vor allem mit Beteiligung des US-Militärs. 2003 haben gleich mehrere US-Software-Unternehmen Übersetzungsprogramme für Arabisch und Paschtu (eine der Sprachen in Afghanistan und Grenzregionen Pakistans) herausgebracht. Ebenfalls 2003 hat die DARPA einen Blind-Wettbewerb für eine unbekannte Ausgangssprache durchgeführt. 2011 wurde das BOLT-Programm gestartet, das zum Ziel hat, die Erforschung der Übersetzung chinesischer und arabischer Texte ins Englische zu fördern.[9][10]
'''

# Set-up translation pipeline

In [8]:
translation_pipeline = pipeline('translation',
                                model=model,
                                tokenizer=tokenizer,
                                src_lang=src_lang,
                                tgt_lang=tgt_lang,
                                max_length=512,
                                device=model.device)

# Translate
Very long sequences could run into the following problems:
*    'Token indices sequence length is longer than the specified maximum sequence length for this model (1320 > 1024).'
*    'Your input_length: 1320 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)'

The model is trained on sentence level and problems are possible with longer sequences. So it's a good idea to split the text.

See paper: 'The maximum sequence length during training is 512 for both the encoder and the decoder.'

The model is restricted to 512 tokens, which means less than 500 words! Text documents should be split into sentences.

In [9]:
translation_pipeline(src_text)

Token indices sequence length is longer than the specified maximum sequence length for this model (1320 > 1024). Running this sequence through the model will result in indexing errors
Your input_length: 1320 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


[{'translation_text': 'В 2016 году, в связи с тем, что перевод на японский язык в сочетании с изучением языка как научной дисциплины (языковедение) впервые открыл конкретный путь к осуществлению этой мечты. История до сегодняшнего дня определяла военный интерес в Пакистане. Одной из самых ранних областей проектов был русско-английский перевод для военных США. Несмотря на его невысокие стандарты, в 2010 году программа перевод на английский язык в США стала более доступной для иностранных специалистов, а также для исследователей из других стран мира. В конце 2006 года, в качестве одного из самых важных примеров этого, впервые сделанного в США, перевод с английского языка на английский язык был осуществлен в Германии. Вместе с тем, в 2006 году, впервые в мире были созданы специальные программы для переводчиков из немецкого языка, такие как Google, Google, Google, Google+ и другие.'}]

# Set-up Sentence Splitter
Use sentence splitter form project [LASER](https://github.com/facebookresearch/LASER), which has been used for NLLB training. This is a small AI-model, which will be downloaded in the background.

In [10]:
from sentence_cleaner_splitter.cleaner_splitter import SentenceSplitClean

sentence_splitter = SentenceSplitClean(src_lang, 'default')

# Split text into sentences
Replace or remove some special chars like zero-width spaces or non-breaking spaces. The NLLB model doesn't understand them.

In [11]:
norm_texts = []
for _, _, line in sentence_splitter(src_text.replace('\u200b', ' ')):
    norm_texts.append(line.strip())

# Translate text
The NLLB model doesn't always map empty lines to empty lines in other languages! This seems to be a bug in the model? We have to postprocess this or insert a different input token for empty lines.

In [12]:
tgt_texts = translation_pipeline(norm_texts)

tgt_texts

[{'translation_text': 'Мечта человечества'},
 {'translation_text': 'Понимание языка, не выучив его, - древняя мечта человечества (строительство Вавилонской башни, числовая интерлингва Дж.'},
 {'translation_text': 'Изобретение компьютеров в сочетании с изучением языка как научной дисциплины (лингвистики) впервые открыло конкретный путь к осуществлению этой мечты.'},
 {'translation_text': 'Весь'},
 {'translation_text': 'История'},
 {'translation_text': 'До сих пор военные интересы определяют путь ТС.'},
 {'translation_text': 'Одним из первых проектов была программа перевода русско-английского для американских военных.'},
 {'translation_text': 'Несмотря на свое анекдотичное низкое качество, программа пользовалась большой популярностью среди американских военных, которые впервые смогли получить хотя бы представление о содержании российских документов без помощи сторонних лиц (переводчиков и переводчиков).'},
 {'translation_text': 'Весь'},
 {'translation_text': 'В 1966 году в докладе ALPAC,

# Show sentences and their matching translation

In [13]:
for s, t in zip(norm_texts, tgt_texts):
  if len(s):
    print(f"{s!r} ---> {t['translation_text']!r}")

'Menschheitstraum' ---> 'Мечта человечества'
'Das Verstehen einer Sprache, ohne sie gelernt zu haben, ist ein alter Menschheitstraum (Turmbau zu Babel, J. Bechers numerische Interlingua, Timerio, Babelfisch, Pfingstwunder, Science-Fiction-Geschichten).' ---> 'Понимание языка, не выучив его, - древняя мечта человечества (строительство Вавилонской башни, числовая интерлингва Дж.'
'Die Erfindung der Computer in Kombination mit der Beschäftigung mit dem Phänomen Sprache als wissenschaftliche Disziplin (Sprachwissenschaft) hat zum ersten Mal einen konkreten Weg zur Erfüllung dieses Traums geöffnet.' ---> 'Изобретение компьютеров в сочетании с изучением языка как научной дисциплины (лингвистики) впервые открыло конкретный путь к осуществлению этой мечты.'
'Geschichte' ---> 'История'
'Bis zum heutigen Tag hat das militärische Interesse den Weg der MÜ entscheidend geprägt.' ---> 'До сих пор военные интересы определяют путь ТС.'
'Eines der frühesten Projekte war ein Russisch-Englisch-Übersetzun